In [ ]:
import tensorflow as tf
import keras
import numpy as np
from keras import backend as K
from keras.models import Model
from keras.layers import (BatchNormalization, Conv1D, Dense, Input, 
    TimeDistributed, Activation, Bidirectional, SimpleRNN, GRU, LSTM, Dropout)

In [ ]:
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", e) 
     
    return mfccsscaled
    
    
# Load various imports 
import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset 
directory = 'Desktop/SD/'
class_label=0

features = []

# Iterate through each sound file and extract the features 
import os
print('OK')
for file in os.listdir(directory):
    print(file)
    for filename in os.listdir(directory+file):
        data = extract_features(directory+'/'+file+'/'+filename)
        features.append([data, class_label])
    class_label=class_label+1

featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files')

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())
# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

xtrain, xtest, ytrain, ytest = train_test_split(X, yy, test_size=0.1, random_state = 42)
print(xtrain.shape)

In [ ]:
xtrain=xtrain.reshape(-1,40,1)
xtest=xtest.reshape(-1,40,1)


In [ ]:
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
def new_sparse_categorical_accuracy(y_true, y_pred):
    y_pred_rank = ops.convert_to_tensor(y_pred).get_shape().ndims
    y_true_rank = ops.convert_to_tensor(y_true).get_shape().ndims
    # If the shape of y_true is (num_samples, 1), squeeze to (num_samples,)
    if (y_true_rank is not None) and (y_pred_rank is not None) and (len(K.int_shape(y_true)) == len(K.int_shape(y_pred))):
        y_true = array_ops.squeeze(y_true, [-1])
    y_pred = math_ops.argmax(y_pred, axis=-1)
        # If the predicted output and actual output types don't match, force cast them
        # to match.
    if K.dtype(y_pred) != K.dtype(y_true):
        y_pred = math_ops.cast(y_pred, K.dtype(y_true))
    return math_ops.cast(math_ops.equal(y_true, y_pred), K.floatx())

        

In [ ]:
input_shape=xtrain.shape
print(input_shape)
from keras.layers import Dropout
model=tf.keras.Sequential([ 
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, input_shape=(40,1),activation='relu',return_sequences=True,implementation=2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(48)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation='softmax')
])
model.build(input_shape)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(xtrain, ytrain, epochs=15, validation_data=(xtest, ytest), verbose=2)